In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from pprint import pprint
from nsepython import *
import numpy as np

In [2]:
#NIFTY URLS
nifty_500_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty500list.csv'
nifty_200_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty200list.csv'
nifty_100_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty100list.csv'
nifty_50_ticker_url = 'https://www1.nseindia.com/content/indices/ind_nifty50list.csv'

In [3]:
# Set universe
universe = nifty_500_ticker_url

In [4]:
# Read CSV & create a tickers df
tickers_file = pd.read_csv(universe)
tickers_df = tickers_file[['Symbol', 'Company Name']]
tickers_list = tickers_df['Symbol']

In [5]:
# News URL
news_url = 'https://ticker.finology.in/company/'

In [11]:
#fetch all available data
article_data = pd.read_csv('./NIFTY_500_Articles.csv', index_col=0)

In [ ]:
'''
# list to store article data
article_data = []
# list to store tickers for which data is unavailable
unavailable_tickers = []
# length of companies
companies_len = len(tickers_list)
tickers_length = companies_len
#days_limit = datetime.datetime.now() - datetime.timedelta(days=30) #only 30 days old or newer articles
print('Fetching Article data..')
for i,ticker in enumerate(tickers_list):
    print(i, ticker)
    url= '{}/{}'.format(news_url, ticker)
    header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(url, headers=header)
    html = BeautifulSoup(response.content, 'lxml')
    news_links = html.select('#newsarticles > a')
    if len(news_links) == 0:
        print('No news found for {}'.format(ticker))
        unavailable_tickers.append(ticker)
        continue
    # for tickers which are not recognized by finology website, it returns home-page. There's also a news section on homepage. so for unrecognized tickers, this function will scrape general financial news instead of ticker specific news
    # var to store article count
    ticker_articles_counter = 0
    for link in news_links:
        art_title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        #if (date_time_obj <= days_limit):
        #     continue
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        article_data.append([ticker, art_title, art_date, art_time])
        ticker_articles_counter += 1
    print('No of articles for {} is {}'.format(ticker, ticker_articles_counter))
    if(ticker_articles_counter==0):
        print('{} added to unavailable list')
        unavailable_tickers.append(ticker)  
'''

In [ ]:
unavailable_tickers

In [ ]:
# run the loop again, but for the tickers for which news could not be fetched.


In [ ]:
#create a df from the news articles list
articles_df = pd.DataFrame(article_data, columns=['Ticker', 'Headline', 'Date', 'Time'])
articles_df

In [ ]:
# get list of tickers for which news was fetched
tickers_list = articles_df['Ticker'].unique()
tickers_list

In [ ]:
#update original tickers_df to remove tickers for which news was not available
tickers_df = tickers_df[tickers_df['Symbol'].isin(tickers_list)].reset_index(drop=True)

In [ ]:
articles_df

In [ ]:
article_data

In [ ]:
article_data.drop(['Unnamed: 0'], axis=1)

In [ ]:
len(article_data['Ticker'].unique())

In [ ]:
# sentiment analysis libraries
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
vader = SentimentIntensityAnalyzer()

In [ ]:
art_scores_df = pd.DataFrame(articles_df['Headline'].apply(vader.polarity_scores).to_list())

In [ ]:
art_scores_df

In [ ]:
# create df with article details and scores merged.
art_scores_df = pd.merge(articles_df, art_scores_df, left_index=True, right_index=True)

In [ ]:
art_scores_df

In [ ]:
#art_scores_df.to_csv('NIFTY_500_Articles.csv')

In [12]:
article_data

,Ticker,Headline,Date,Time,neg,neu,pos,compound
0,ADANIENT,"Adani Enterprises raises Rs 5,985 crore from a...",2023/01/27,11:28,0.0,1.000,0.000,0.0000
1,ADANIENT,Adani Enterprises submits media statement,2023/01/25,16:33,0.0,1.000,0.000,0.0000
2,ADANIENT,Adani group planning to spin off businesses of...,2023/01/23,11:11,0.0,1.000,0.000,0.0000
3,ADANIENT,Adani Enterprises’ arm incorporates JV Company,2023/01/21,17:16,0.0,1.000,0.000,0.0000
4,ADANIENT,Adani Enterprises planning to enter water segment,2023/01/20,11:06,0.0,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
2429,WIPRO,Wipro enters into partnership with Finastra,2022/09/20,17:58,0.0,1.000,0.000,0.0000
2430,WIPRO,Wipro informs about press release,2022/09/20,09:37,0.0,1.000,0.000,0.0000
2431,WIPRO,Wipro launches @nowStudio in Brazil,2022/09/07,16:30,0.0,1.000,0.000,0.0000
2432,WIPRO,Wipro expands collaboration with Palo Alto Net...,2022/09/06,18:04,0.0,0.811,0.189,0.1027


In [13]:
ticker_metadata = pd.read_csv('./ticker_metadata.csv', index_col=0)

In [ ]:
'''
# Get Company,Sector,Industry,mCap Data
ticker_meta = []
new_length = len(tickers_list)
#new_length = 10
print('Fetching ticker meta data')
# whether to get only data available in tickers_list or the whole universe bc even if we take whole universe, we will later merge the dfs ticker_meta and art_scores_df
# but since we're planning on not to fetch industry and sector data that often, its better to take the universe meta

for i, ticker in enumerate(tickers_list):
    meta = nse_eq(ticker)
    print(i, ticker)
    try:
        sector = meta['industryInfo']['macro']
    except KeyError:
        print('{} sector info is not available'.format(ticker))
        sector = np.nan
        industry = np.nan
        mCap = np.nan
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
        continue
    try:
        industry = meta['industryInfo']['industry']
    except KeyError:
        print('{} industry info is not available'.format(ticker))
        industry = np.nan
        mCap = np.nan
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
        continue
    try:
        mCap = round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2)
    except KeyError:
        print('{} mCap data is not available'.format(ticker))
        mCap = np.nan
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
        continue
    try:
        companyName = meta['info']['companyName']
    except KeyError:
        print('{} company Name is not available'.format(ticker))
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
    ticker_meta.append([ticker, sector, industry, mCap, companyName])

'''

In [ ]:
xc_indices = pd.read_csv('XC-tickers.csv')
xc_indices.columns = ['Ticker', 'XC-Sector', 'XC-Industry']

In [ ]:
xc_indices

In [ ]:
ticker_meta_df = pd.DataFrame(ticker_meta, columns=['Ticker', 'Sector', 'Industry', 'Market Cap', 'Company Name'])

In [ ]:
ticker_meta_df

In [ ]:
ticker_metadata = pd.merge(ticker_meta_df, xc_indices, on='Ticker')

In [14]:
ticker_metadata

,Ticker,Sector,Industry,Market Cap,Company Name,XC-Sector,XC-Industry
0,ADANIENT,Commodities,Metals & Minerals Trading,3863.41,Adani Enterprises Limited,XC - INDS,XC - CAG
1,ADANIPORTS,Services,Transport Infrastructure,1540.50,Adani Ports and Special Economic Zone Limited,XC - INDS,XC - SHP
2,APOLLOHOSP,Healthcare,Healthcare Services,607.40,Apollo Hospitals Enterprise Limited,XC - HECA,XC - HCS
3,ASIANPAINT,Consumer Discretionary,Consumer Durables,2661.77,Asian Paints Limited,XC - COST,XC - HPP
4,AXISBANK,Financial Services,Banks,2744.42,Axis Bank Limited,XC - FNCL,XC - PVT
5,BAJAJ-AUTO,Consumer Discretionary,Automobiles,1051.87,Bajaj Auto Limited,XC - CODI,XC - ATO
6,BAJFINANCE,Financial Services,Finance,3509.01,Bajaj Finance Limited,XC - FNCL,XC - NFC
7,BAJAJFINSV,Financial Services,Finance,1256.11,Bajaj Finserv Limited,XC - FNCL,XC - NFC
8,BPCL,Energy,Petroleum Products,752.19,Bharat Petroleum Corporation Limited,XC - ENRG,XC - REF
9,BHARTIARTL,Telecommunication,Telecom - Services,4327.91,Bharti Airtel Limited,XC - COSE,XC - TEL


In [ ]:
#ticker_metadata.to_csv('ticker_matadata.csv')

In [ ]:
ticker_metadata.drop(['Unnamed: 0'], axis=1)

In [ ]:
article_data = article_data.drop(['Unnamed: 0'], axis=1)
ticker_metadata = ticker_metadata.drop(['Unnamed: 0'], axis=1)

In [ ]:
article_score_summary = article_data.groupby('Ticker').mean()

In [ ]:
article_score_summary

In [ ]:
pd.merge(article_score_summary, ticker_metadata,  on='Ticker', how='inner')

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

In [18]:
ticker_scores = article_data.groupby('Ticker').mean().reset_index()

C:\Users\singh\AppData\Local\Temp\ipykernel_12688\1282117467.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ticker_scores = article_data.groupby('Ticker').mean().reset_index()


In [19]:
ticker_scores

,Ticker,neg,neu,pos,compound
0,ADANIENT,0.00192,0.966740,0.031340,0.049380
1,ADANIPORTS,0.00000,0.893660,0.106340,0.187054
2,APOLLOHOSP,0.05478,0.835900,0.109340,0.102598
3,ASIANPAINT,0.02206,0.935940,0.042000,0.035954
4,AXISBANK,0.00000,0.937400,0.062600,0.117648
5,BAJAJ-AUTO,0.00000,0.972100,0.027900,0.047072
6,BAJAJFINSV,0.02362,0.906000,0.070380,0.063730
7,BAJFINANCE,0.00792,0.887640,0.104440,0.152188
8,BHARTIARTL,0.04066,0.944100,0.015240,-0.011274
9,BPCL,0.03930,0.880440,0.080260,0.069378


In [22]:
final_df = pd.merge(ticker_metadata, ticker_scores, on='Ticker', how='inner')

In [24]:
final_df.columns = ['Symbol','Macro-Sector','Industry', 'Market Cap (Billion Rs)','Company Name','XC-Sector', 'XC-Industry', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']

In [25]:
final_df

,Symbol,Macro-Sector,Industry,Market Cap (Billion Rs),Company Name,XC-Sector,XC-Industry,Negative,Neutral,Positive,Sentiment Score
0,ADANIENT,Commodities,Metals & Minerals Trading,3863.41,Adani Enterprises Limited,XC - INDS,XC - CAG,0.00192,0.966740,0.031340,0.049380
1,ADANIPORTS,Services,Transport Infrastructure,1540.50,Adani Ports and Special Economic Zone Limited,XC - INDS,XC - SHP,0.00000,0.893660,0.106340,0.187054
2,APOLLOHOSP,Healthcare,Healthcare Services,607.40,Apollo Hospitals Enterprise Limited,XC - HECA,XC - HCS,0.05478,0.835900,0.109340,0.102598
3,ASIANPAINT,Consumer Discretionary,Consumer Durables,2661.77,Asian Paints Limited,XC - COST,XC - HPP,0.02206,0.935940,0.042000,0.035954
4,AXISBANK,Financial Services,Banks,2744.42,Axis Bank Limited,XC - FNCL,XC - PVT,0.00000,0.937400,0.062600,0.117648
5,BAJAJ-AUTO,Consumer Discretionary,Automobiles,1051.87,Bajaj Auto Limited,XC - CODI,XC - ATO,0.00000,0.972100,0.027900,0.047072
6,BAJFINANCE,Financial Services,Finance,3509.01,Bajaj Finance Limited,XC - FNCL,XC - NFC,0.00792,0.887640,0.104440,0.152188
7,BAJAJFINSV,Financial Services,Finance,1256.11,Bajaj Finserv Limited,XC - FNCL,XC - NFC,0.02362,0.906000,0.070380,0.063730
8,BPCL,Energy,Petroleum Products,752.19,Bharat Petroleum Corporation Limited,XC - ENRG,XC - REF,0.03930,0.880440,0.080260,0.069378
9,BHARTIARTL,Telecommunication,Telecom - Services,4327.91,Bharti Airtel Limited,XC - COSE,XC - TEL,0.04066,0.944100,0.015240,-0.011274


In [27]:
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

In [28]:
print('Generating Plots')
fig = px.treemap(
    final_df, path=[px.Constant('Nifty 500'), 'XC-Sector', 'XC-Industry', 'Symbol'], values='Market Cap (Billion Rs)', color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)

Generating Plots


--------------------------

In [ ]:
test_tickers = ['3MINDIA', 'APLAPOLLO', 'ABBOTINDIA', 'ABSLAMC', 'ALKYLAMINE', 'AMARAJABAT', 'AMBER', 'APOLLOTYRE', 'APTUS', 'ASTRAZEN', 'BEML', 'BAJAJELEC', 'BALKRISIND', 'BATAINDIA', 'BAYERCROP', 'BERGEPAINT', 'BHARATRAS', 'BLUEDART', 'BRITANNIA', 'MAPMYINDIA', 'CESC', 'CAMPUS', 'CEATLTD', 'CDSL', 'CENTURYPLY', 'CHOLAHLDNG', 'COLPAL', 'CUMMINSIND', 'DEVYANI', 'DIVISLAB', 'DIXON', 'EIHOTEL', 'EPL', 'EMAMILTD', 'NYKAA', 'FACT', 'FINEORG', 'GAIL', 'GICRE', 'GODFRYPHLP', 'GUJALKALI', 'GUJGASLTD', 'GSPL', 'HLEGLAS', 'HIKAL', 'HGS', 'HAL', 'HONAUT', 'HUDCO', 'IFBIND', 'INDIACEM', 'IBULHSGFIN', 'IRFC', 'IGL', 'IPCALAB', 'JBCHEPHARM', 'KNRCON', 'KPITTECH', 'KANSAINER', 'L&TFH', 'LAXMIMACH', 'LUXIND', 'MRF', 'MGL', 'M&MFIN', 'M&M', 'MAHLOG', 'MAZDOCK', 'MEDPLUS', 'METROPOLIS', 'MSUMI', 'MUTHOOTFIN', 'NLCINDIA', 'NATIONALUM', 'NAVINFLUOR', 'NESTLEIND', 'NAM-INDIA', 'NUVOCO', 'OBEROIRLTY', 'OIL', 'OLECTRA', 'PCBL', 'PATANJALI', 'PFIZER', 'POLYPLEX', 'PFC', 'PRESTIGE', 'PRINCEPIPE', 'PGHL', 'RAIN', 'RAINBOW', 'RATNAMANI', 'REDINGTON', 'RBA', 'SIS', 'SRF', 'SHILPAMED', 'SAIL', 'SUMICHEM', 'SUNTV', 'SUNDARMFIN', 'SUPREMEIND', 'SYMPHONY', 'TCNSBRANDS', 'TTML', 'TEJASNET', 'THERMAX', 'THYROCARE', 'TORNTPHARM', 'TIINDIA', 'UPL', 'UTIAMC', 'VIPIND', 'VAIBHAVGBL', 'VBL', 'MANYAVAR', 'VIJAYA', 'VOLTAS', 'WHIRLPOOL', 'WOCKPHARMA', 'ZFCVINDIA', 'ZEEL', 'ZENSARTECH', 'ZOMATO', 'ECLERX']

In [ ]:
len(test_tickers)

In [ ]:
news_url = 'https://ticker.finology.in/company/'
# list to store article data
article_data = []
unavailable_tickers = []
# list to store tickers for which data is unavailable
# length of companies
companies_len = len(test_tickers)
tickers_length = companies_len
#days_limit = datetime.datetime.now() - datetime.timedelta(days=30) #only 30 days old or newer articles
print('Fetching Article data..')
for i in range(tickers_length):
    print(i, test_tickers[i])
    url= '{}/{}'.format(news_url, test_tickers[i])
    header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(url, headers=header)
    html = BeautifulSoup(response.content, 'lxml')
    news_links = html.select('#newsarticles > a')
    if len(news_links) == 0:
        print('No news found for {}'.format(test_tickers[i]))
        unavailable_tickers.append(test_tickers[i])
        continue
    # for tickers which are not recognized by finology website, it returns home-page. There's also a news section on homepage. so for unrecognized tickers, this function will scrape general financial news instead of ticker specific news
    # var to store article count
    ticker_articles_counter = 0
    for link in news_links:
        art_title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        #if (date_time_obj <= days_limit):
        #     continue
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        article_data.append([test_tickers[i], art_title, art_date, art_time])
        ticker_articles_counter += 1
    if(ticker_articles_counter==0):
        print('Article length = 0 for {}'.format(test_tickers[i]))
        unavailable_tickers.append(tickers_list[i])  

In [ ]:
pprint(article_data)

--------------------------

In [ ]:
url= 'https://ticker.finology.in/company/EMAMILTD'
header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
response = requests.get(url, headers=header)
html = BeautifulSoup(response.content, 'lxml')

In [ ]:
news_links = html.select('#newsarticles > a')

In [ ]:
news_links